In [5]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
from ipywidgets import interact
import pandas as pd
import numpy as np
import math
import time
# output to the IPython Notebook
output_notebook()
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm
import bisect
from sklearn.neighbors.kde import KernelDensity
from scipy.stats import genextreme as gev
import scipy.stats as ss
from scipy.stats import gamma
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
# import seaborn
import seaborn as sns
# settings for seaborn plotting style
sns.set(color_codes=True)
# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(4.5,3)})

Loading BokehJS ...

# 1- Input data

In [6]:
# Some characteristics for plots
plt.rc('legend',**{'fontsize':16})
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 

In [7]:
seed_number = 22331
np.random.seed(seed_number)
Pit_generation_rate = 80 # assumption
ScalePar = 3.5    # Assumed scale parameter of the gamma process
k  = 0.12 # Ossai 2016 paper
nu = 0.771  # Ossai 2016 paper
ILI_no = 3+1   # I aasumed there are 3 ILI datasets, plus the prediction time
ILI_info = pd.DataFrame(0, index = range(1, ILI_no + 1), columns = range(5)).astype(float)
ILI_info.columns = ['ILI_time', 'Actual_Pit_No', 'Estimated_Pit_No', 'KLD', 'ChiSq']
ILI_info.index = ['ILI_1', 'ILI_2', 'ILI_3', 'Prediction Time']
ILI_info.ILI_time = [30, 35, 40, 50] # I assumed that the first ILI happened 30 years after starting operation and so on
ILI_info.Actual_Pit_No = [np.random.poisson(Pit_generation_rate*ILI_info.ILI_time[0]),\
                       np.random.poisson(Pit_generation_rate*(ILI_info.ILI_time[1]-ILI_info.ILI_time[0])),\
                       np.random.poisson(Pit_generation_rate*(ILI_info.ILI_time[2]-ILI_info.ILI_time[1])),\
                       np.random.poisson(Pit_generation_rate*(ILI_info.ILI_time[3]-ILI_info.ILI_time[2]))]
ILI_info

,ILI_time,Actual_Pit_No,Estimated_Pit_No,KLD,ChiSq
ILI_1,30,2374,0.0,0.0,0.0
ILI_2,35,414,0.0,0.0,0.0
ILI_3,40,381,0.0,0.0,0.0
Prediction Time,50,790,0.0,0.0,0.0


In [8]:
PWT = 8.41 # mm, pipe wall thickness
# Detection tool's specifications  
detection_threshold = 0.1 * PWT # from Dann and Maes 2018
credible_pit_depth_threshold  = 0.2 * PWT # from Dann and Maes 2018
a_ILI = 2.04/100*PWT  # from Zhang and Zhou 2013
b_ILI = 0.97  # from Zhang and Zhou 2013
ILI_percent_error = 5.97/100   # from Zhang and Zhou 2013
ILI_error_STD = (ILI_percent_error * PWT) # from Zhang and Zhou 2013

# 2 - Generating synthetic actual and measured depth

In [9]:
# defining the time steps given start and end point of each gamma process
t0 = 0;
h = 1 # time step
def time_info (start, end):
    T = end - start
    N = np.int(T / h) + 1; # No. of time steps
    t = np.linspace(t0, T, N)
    return t

In [10]:
# figure specifications for Bokeh plots
p = figure(x_axis_label='t (year)',\
           y_axis_label='depth (mm)', plot_height = 700, plot_width = 700)      
p.xaxis.axis_line_width = 3
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.axis_line_width = 3 
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.axis_label_text_font_size  = "20pt"
p.yaxis.axis_label_text_font_size  = "20pt"
p.axis.minor_tick_in  = 2
p.axis.minor_tick_out = 5
p.axis.axis_label_text_alpha=0.9
p.legend.label_text_font_size = "20pt"

In [11]:
# Generating synthetic actual and measured depth for those pits that are initiated before ILI 1 at time zero
np.random.seed(seed_number)
t1 = time_info (0, ILI_info.ILI_time[3])
# The actual depth for those pits that are initiated before ILI_1
Initiation_1_Actual_Depth = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[0]+1), columns = range(ILI_no))
# POD_1 is probability of detection based on the actual depth
POD_1 = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[0]+1), columns = range(ILI_no))
Measured_Depth = pd.DataFrame(0, index = range(1, 2), columns = range(10)) # 10 = 4 (no of measurement for the first set)\
 # + 3 (no of measurements for the second set) + 2 (..third.) + 1 (..forth.)
ILI_Index = 1
for i in tqdm(range(1, ILI_info.Actual_Pit_No[ILI_Index-1]+1)):
    for j in range(0, ILI_no):
        g = np.random.gamma( k*(ILI_info.ILI_time[j]-np.random.uniform(0, ILI_info.ILI_time[0]))**nu, scale = ScalePar) ; # degradation increment
        Initiation_1_Actual_Depth.loc[i, j] = g # storing the actual depth at ILI time and prediction time
        POD_1.loc[i, j] = 1 - np.exp(- Initiation_1_Actual_Depth.loc[i, j] / detection_threshold) 
        if POD_1.loc[i, j] > np.random.uniform(): #  if a pit will be detected or not
            Measured_Depth.loc[i, j] = a_ILI + b_ILI * Initiation_1_Actual_Depth.loc[i, j] \
            + np.random.normal (0, ILI_error_STD)
        else:
            Measured_Depth.loc[i, j] = 'NaN'
        
p.line(t1, k*(t1-t0)**nu*ScalePar , line_width = 2, legend ='Before ILI 1 : ' + str (ILI_info.Actual_Pit_No[0]),\
       color = 'yellow')


KeyboardInterrupt: 

In [ ]:
# Visual comparison of samples data and actual distribution
tt = np.linspace(t0, 50, 1000)
bins_no = 100
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth (mm)', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)
plt.plot(tt, gamma.pdf(tt, k*(ILI_info.ILI_time[0]-t0)**nu, scale = ScalePar), c='green', linestyle='--', alpha=0.5,\
         linewidth=3, label='Actual pit depth dist. ILI_1') 
plt.hist(Initiation_1_Actual_Depth[0], bins_no, color = 'green',  histtype='step', normed=True, linewidth=2,\
          label='Sampled actual depth at ILI_1');
pltEndPoint = 14
plt.xlim([0, pltEndPoint]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
# Generating synthetic actual and measured depth for those pits that are initiated between ILI 1 and ILI 2
np.random.seed(seed_number)
t2 = time_info (ILI_info.ILI_time[0], ILI_info.ILI_time[3])
Initiation_2_Actual_Depth = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[1]+1), columns = range(ILI_no-1))
POD_2 = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[1]+1), columns = range(1, ILI_no-1))
ILI_Index = 2
for i in range (1, ILI_info.Actual_Pit_No[ILI_Index-1]+1):
    for j in range(1, ILI_no):
        g = np.random.gamma(k*(ILI_info.ILI_time[j]-np.random.uniform(ILI_info.ILI_time[0], ILI_info.ILI_time[1]))**nu, scale = ScalePar) ;
        Initiation_2_Actual_Depth.loc[i, j-1] = g
        POD_2.loc[i, j] = 1 - np.exp(- Initiation_2_Actual_Depth.loc[i, j-1] / detection_threshold) 
        if POD_2.loc[i, j] > np.random.uniform():
            Measured_Depth.loc[i, j + 3] = a_ILI + b_ILI * Initiation_2_Actual_Depth.loc[i, j-1] \
            + np.random.normal (0, ILI_error_STD)
        else:
            Measured_Depth.loc[i, j + 3] = 'NaN'
        
p.line(ILI_info.ILI_time[0] + t2, k*(t2-t0)**nu*ScalePar , line_width = 2, legend ='Between ILI 1 & 2 : ' \
       + str (ILI_info.Actual_Pit_No[1]), color = 'yellow')

In [ ]:
# Visual comparison of samples data and actual distribution
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth(mm)', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)
plt.plot(tt, gamma.pdf(tt, k*(ILI_info.ILI_time[3]-ILI_info.ILI_time[0])**nu, scale = ScalePar), c='green', linestyle='--', alpha=0.5,  linewidth=3,\
         label='Actual pit depth dist. ILI_2_2') 
plt.hist(Initiation_2_Actual_Depth[2], bins_no, color = 'green',  histtype='step', normed=True, linewidth=2,\
         label='Sampled actual depth at ILI_2_2');

plt.xlim([0, 18]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
# Generating synthetic actual and measured depth for those pits that are initiated between ILI 2 and ILI 3
np.random.seed(seed_number)
t3 = time_info (ILI_info.ILI_time[1], ILI_info.ILI_time[3])
Initiation_3_Actual_Depth = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[2]+1), columns = range(1, ILI_no-2))
POD_3 = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[2]+1), columns = range(1, ILI_no-2))
ILI_Index = 3
for i in tqdm(range(1, ILI_info.Actual_Pit_No[ILI_Index-1]+1)):
    for j in range (2, ILI_no):
        g = np.random.gamma(k*(ILI_info.ILI_time[j]-np.random.uniform(ILI_info.ILI_time[1], ILI_info.ILI_time[2]))**nu, scale = ScalePar) ;
        Initiation_3_Actual_Depth.loc[i, j-2] = g
        POD_3.loc[i, j] = 1 - np.exp(- Initiation_3_Actual_Depth.loc[i, j-2] / detection_threshold) 
        if POD_3.loc[i, j] > np.random.uniform():
            Measured_Depth.loc[i, j + 5] = a_ILI + b_ILI * Initiation_3_Actual_Depth.loc[i, j-2] \
            + np.random.normal (0, ILI_error_STD)
        else:
            Measured_Depth.loc[i, j + 5] = 'NaN'                
p.line(ILI_info.ILI_time[1] + t3, k*(t3-t0)**nu*ScalePar , line_width = 2, legend ='Between ILI 2 & 3 : ' \
       + str (ILI_info.Actual_Pit_No[2]), color = 'yellow')

In [ ]:
# Visual comparison of samples data and actual distribution

plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth(mm)', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)
plt.plot(tt, gamma.pdf(tt, k*(ILI_info.ILI_time[3]-ILI_info.ILI_time[1])**nu, scale = ScalePar), c='green', linestyle='--', alpha=0.5,  linewidth=3,\
         label='Actual pit depth dist. ILI_3') 
plt.hist(Initiation_3_Actual_Depth[1], bins_no, color = 'green',  histtype='step', normed=True, linewidth=2,\
         label='Sampled actual depth at ILI_3');

plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
# Generating synthetic actual and measured depth for those pits that are initiated between ILI 3 and prediction time
np.random.seed(seed_number)
t4 = time_info (ILI_info.ILI_time[2], ILI_info.ILI_time[3])
Prediction_Actual_Depth = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[3]+1), columns = range(ILI_no-3))
POD_4 = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[3]+1), columns = range(ILI_no-2))
ILI_Index = 4 # This is for prediction and it is not an ILI
for i in tqdm(range(1, ILI_info.Actual_Pit_No[ILI_Index-1]+1)):
    for j in range (3, ILI_no):
        g = np.random.gamma(k*(ILI_info.ILI_time[j]-np.random.uniform(ILI_info.ILI_time[2],ILI_info.ILI_time[3]))**nu, scale = ScalePar) ;           
        Prediction_Actual_Depth.loc[i, j-3] = g
        POD_4.loc[i, j] = 1 - np.exp(- Prediction_Actual_Depth.loc[i, j-3] / detection_threshold) 
        if POD_4.loc[i, j] > np.random.uniform():
            Measured_Depth.loc[i, j + 6] = a_ILI + b_ILI * Prediction_Actual_Depth.loc[i, j-3] \
                + np.random.normal (0, ILI_error_STD)
        else:
            Measured_Depth.loc[i, j + 6] = 'NaN'                
p.line(ILI_info.ILI_time[2] + t4, k*(t4-t0)**nu*ScalePar , line_width = 2, legend ='After ILI 3 : ' \
       + str (ILI_info.Actual_Pit_No[3]), color = 'yellow')
show(p) 

In [ ]:
# Visual comparison of samples data and actual distribution

plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth(mm)', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)
plt.plot(tt, gamma.pdf(tt, k*(ILI_info.ILI_time[3]-ILI_info.ILI_time[2])**nu, scale = ScalePar), c='green', linestyle='--', alpha=0.5,  linewidth=3,\
         label='Actual pit depth dist. Prediction') 
plt.hist(Prediction_Actual_Depth[0], bins_no, color = 'green',  histtype='step', normed=True, linewidth=2,\
         label='Sampled actual depth at Prediction');
plt.xlim([0, 8]);

plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
# Remove the NaN values from measurements 
Cleaned_Measured_Depth_Simulated = pd.DataFrame(0, index = range(1, ILI_info.Actual_Pit_No[0]+1), columns = range(10)) 
No_Detected_Pits = pd.DataFrame(0, index = range(1, 2), columns = range(10))
for q in range(0,4):
    if q == 0: # Columns 0 to 3 of Measured_Depth are belong to those pits generated before ILI 1
        v = 0
        u = 4
    elif q == 1: # Columns 4 to 6 of Measured_Depth are belong to those pits generated beteween ILI 1 and 2
        v = 4
        u = 7
    elif q == 2: # Columns 7 to 8 of Measured_Depth are belong to those pits generated beteween ILI 2 and 3
        v = 7
        u = 9
    elif q == 3: # Column 9 of Measured_Depth are belong to those pits generated beteween ILI 3 and prediction time
        v = 9
        u = 10        
    for j in range (v, u):
        ii = 1
        for i in range(1, ILI_info.Actual_Pit_No[q]+1):
            if isinstance(Measured_Depth.loc[i,j], str) == False:                
                Cleaned_Measured_Depth_Simulated.loc[ii,j] = Measured_Depth.loc[i,j]
                No_Detected_Pits.loc[1, j] = No_Detected_Pits.loc[1, j] + 1
                ii = ii + 1

In [ ]:
Cleaned_Measured_Depth_Simulated.columns = ['Y_1_1', 'Y_2_1', 'Y_3_1', 'Y_4_1', 'Y_2_2', 'Y_3_2', 'Y_4_2',\
                                 'Y_3_3', 'Y_3_4', 'Y_4_4']

In [ ]:
Cleaned_Measured_Depth_Simulated.head()

In [ ]:
# Save the generated synthetic data
Measured_Depth.to_csv('Measured_Depth.csv')
Initiation_1_Actual_Depth.to_csv('Initiation_1_Actual_Depth.csv')
Initiation_2_Actual_Depth.to_csv('Initiation_2_Actual_Depth.csv')
Initiation_3_Actual_Depth.to_csv('Initiation_3_Actual_Depth.csv')
Prediction_Actual_Depth.to_csv('Prediction_Actual_Depth.csv')
Cleaned_Measured_Depth_Simulated.to_csv('Cleaned_Measured_Depth_Simulated.csv')

In [ ]:
m_1 = No_Detected_Pits[0] # Number of detected pits at ILI 1
m_2 = No_Detected_Pits[1] + No_Detected_Pits[4] # Number of detected pits at ILI 2
m_3 = No_Detected_Pits[2] + No_Detected_Pits[5] + No_Detected_Pits[7] # Number of detected pits at ILI 3
m_4 = No_Detected_Pits[3] + No_Detected_Pits[6] + No_Detected_Pits[8] + No_Detected_Pits[9] # Number of detected pits at prediction time

# 2-1-Add POFC

In this section false call error is added to consider POFC in synthetic data generation.

# 2-1-1 ILI_1

In [ ]:
# Number of true calls at each ILI
No_TrueCalled_Pits = pd.DataFrame(0, index = range(1, 2), columns = range(3))

In [ ]:
# Comparing the PMF of the actual depth of existing pits and the PMF of measurements of detected pits at ILI_1
frequency_info_1 = pd.DataFrame(0, index = range(1, bins_no+1), columns = range(6)).astype(float)
frequency_info_1.columns = ['actual_N','detected_N', 'reported_N', 'estimated_N_WO_POD', 'estimated_N_W_POD',\
                            'rounded_estimated_N_W_POD']
bins_start_1 = np.int(np.min(Initiation_1_Actual_Depth[0]))
bins_end_1   = np.int(np.max(Initiation_1_Actual_Depth[0]))
# find the bins intervals and the bins frequencies
(actual_N_1, bins_1, patches) = plt.hist(Initiation_1_Actual_Depth[0], bins_no,\
                                     range=(bins_start_1, bins_end_1),\
                                     histtype='step', normed=False, linewidth=2, color = 'blue',\
                                     label='Distribution of actual samples ILI_1_1');
# find the bins intervals and the bins frequencies
(detected_N, bins_1, patches1) = plt.hist(Cleaned_Measured_Depth_Simulated.Y_1_1[1:1642], bins_no,\
                                          range=(bins_start_1, bins_end_1),\
                                                 histtype='step', normed=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');
frequency_info_1.actual_N = actual_N_1
frequency_info_1.detected_N = detected_N

In [ ]:
# Estimate the number of reported pits given synthetic detected pits 
for i in range (1, bins_no+1):
    POFC = np.exp(-(bins_1[i-1]+bins_1[i])/2/credible_pit_depth_threshold)  
    frequency_info_1.reported_N[i] = frequency_info_1.detected_N[i]/(1-POFC)

In [ ]:
# find the distribution of R_1_1
np.random.seed(seed_number)
R_1_1 = pd.DataFrame(0, index = range(0, np.int(np.sum(frequency_info_1.reported_N))), columns = range(1)).astype(float) 
# Part of R_1_1 is equal to the truly detected pits
for i in range(0, np.int(m_1)):
    R_1_1[0][i] = Cleaned_Measured_Depth_Simulated.Y_1_1[i+1]
jj = 0
# for those bins that frequency of reported pits is higher than the detected pits, generate random number between lower and 
# upper bound of that bin
for i in range(1, bins_no+1):      
    if frequency_info_1.reported_N[i] > frequency_info_1.detected_N[i]:        
        for j in range(1, np.int(frequency_info_1.reported_N[i] - frequency_info_1.detected_N[i])+1):          
            R = np.random.uniform(bins_1[i-1], bins_1[i])
            R_1_1[0][np.int(m_1) + jj] = R
            jj = jj + 1     
R_1_1.to_csv('R_1_1.csv')

In [ ]:
np.random.seed(seed_number)
# Fliter true calls measurements from reported measurements
Cleaned_Measured_Depth = pd.DataFrame(0, index = range(0, 5000), columns = range(4)).astype(float) 
Cleaned_Measured_Depth.columns = ['Y_1', 'Y_2', 'Y_3', 'Y_4']
No_TrueCalled_Pits[0][1] = 0
for item in R_1_1[0]:           
    if np.random.uniform(0,1) > np.exp(-item/credible_pit_depth_threshold):         
        Cleaned_Measured_Depth.Y_1[No_TrueCalled_Pits[0][1]] = item        
        No_TrueCalled_Pits[0][1] = No_TrueCalled_Pits[0][1] + 1

In [ ]:
# Comapring histograms
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth(mm)', fontsize = 20)
plt.ylabel('PMF', fontsize = 20)
plt.hist(R_1_1[0], bins_no, color = 'blue', range=(bins_start_1, bins_end_1), histtype='step', density=False, linewidth=2,\
         label='Reported measurements');  
plt.hist(Cleaned_Measured_Depth.Y_1[0:No_TrueCalled_Pits[0][1]], bins_no, color = 'red', range=(bins_start_1, bins_end_1),\
         histtype='step', density=False, linewidth=1,\
         label='True Call measurements'); 
plt.xlim(bins_start_1, 30);

plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

# 2-1-1 ILI_2

In [ ]:
# Y_2_d is the Cleaned simulated measured depth at ILI_2
Y_2_d = pd.DataFrame(pd.concat([Cleaned_Measured_Depth_Simulated.Y_2_1[0:No_Detected_Pits.loc[1][1]],\
                                         Cleaned_Measured_Depth_Simulated.Y_2_2[0:No_Detected_Pits.loc[1][4]]], axis=0,\
                                        join='outer', ignore_index=True, keys=None, levels=None, names=None,\
                                        verify_integrity=False, copy=True))

In [ ]:
# Comparing the histograms and finding the bins intervals and frequencies
frequency_info_2 = pd.DataFrame(0, index = range(1, bins_no+1), columns = range(6)).astype(float)
frequency_info_2.columns = ['actual_N','detected_N', 'reported_N', 'estimated_N_WO_POD', 'estimated_N_W_POD',\
                            'rounded_estimated_N_W_POD']
bins_start_2 = np.int(np.min(pd.concat([Initiation_2_Actual_Depth[0], Initiation_1_Actual_Depth[1]])))
bins_end_2   = np.int(np.max(pd.concat([Initiation_2_Actual_Depth[0], Initiation_1_Actual_Depth[1]])))
(actual_N_2, bins_2, patches) = plt.hist(pd.concat([Initiation_2_Actual_Depth[0], Initiation_1_Actual_Depth[1]]), bins_no,\
                                     range=(bins_start_2, bins_end_2),\
                                     histtype='step', normed=False, linewidth=2, color = 'blue',\
                                     label='Distribution of actual samples ILI_2')
(detected_N_2, bins_2, patches2) = plt.hist(Y_2_d[0], bins_no, range=(bins_start_2, bins_end_2),\
                                                 histtype='step', normed=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');


frequency_info_2.detected_N = detected_N_2
frequency_info_2.actual_N = actual_N_2

In [ ]:
# Estimate the number of reported pits given synthetic detected pits
for i in range (1, bins_no+1):
    POFC = np.exp(-(bins_2[i-1]+bins_2[i])/2/credible_pit_depth_threshold)  
    frequency_info_2.reported_N[i] = frequency_info_2.detected_N[i]/(1-POFC)

In [ ]:
# estimate the actual depth of reported pits and find the distribution of R_2
np.random.seed(seed_number)
R_2 = pd.DataFrame(0, index = range(0, np.int(np.sum(frequency_info_2.reported_N))), columns = range(1)).astype(float) 
for i in range(0, np.int(m_2)):
    R_2[0][i] = Y_2_d[0][i]
# for those bins that frequency of reported pits is higher than the detected pits, generate random number between lower and 
# upper bound of that bin
jj = 0
for i in range(1, bins_no+1):      
    if frequency_info_2.reported_N[i] > frequency_info_2.detected_N[i]:        
        for j in range(1, np.int(frequency_info_2.reported_N[i] - frequency_info_2.detected_N[i])+1):          
            R = np.random.uniform(bins_2[i-1], bins_2[i])
            R_2[0][np.int(m_2) + jj] = R
            jj = jj + 1     
R_2.to_csv('R_2.csv')

In [ ]:
# Fliter true calls measurements from reported measurements
np.random.seed(seed_number)
# Filter false calls from reported data 
No_TrueCalled_Pits[1][1] = 0
for item in R_2[0]:           
    if np.random.uniform(0,1) > np.exp(-item/credible_pit_depth_threshold):
        Cleaned_Measured_Depth.Y_2[No_TrueCalled_Pits[1][1]] = item
        No_TrueCalled_Pits[1][1] = No_TrueCalled_Pits[1][1] + 1
print(No_TrueCalled_Pits[1][1])

In [ ]:
# Comparison
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)


dd = pd.DataFrame(pd.concat([Initiation_1_Actual_Depth[1], Initiation_2_Actual_Depth[0]], axis=0,\
                                        join='outer', ignore_index=True, keys=None, levels=None, names=None,\
                                        verify_integrity=False, copy=True))
plt.hist(dd[0], bins_no, color = 'green', range=(bins_start_2, bins_end_2),\
          histtype='step', density=True, linewidth=2, label='Actual depth at 1_1');
plt.hist(Y_2_d[0], bins_no, color = 'red', range=(bins_start_2, bins_end_2), histtype='step', density=True, linewidth=2,\
         label='Detected');  
plt.hist(R_2[0], bins_no, color = 'blue', range=(bins_start_2, bins_end_2), histtype='step', density=True, linewidth=2,\
         label='Reported');  
plt.hist(Cleaned_Measured_Depth.Y_2[0:No_TrueCalled_Pits[1][1]], bins_no, color = 'yellow', range=(bins_start_2, bins_end_2),\
         histtype='step', density=True, linewidth=2,\
         label='TrueCalls'); 
plt.xlim(bins_start_1, bins_end_1);

plt.title('Updating Hyper-parameters' , fontsize = 16)
plt.legend(loc='upper right');

# 2-1-2 ILI_3

In [ ]:
# Y_3_d is the Cleaned simulated measured depth at ILI_3
Y_3_d = pd.DataFrame(pd.concat([Cleaned_Measured_Depth_Simulated.Y_3_1[0:No_Detected_Pits.loc[1][2]],\
                                Cleaned_Measured_Depth_Simulated.Y_3_2[0:No_Detected_Pits.loc[1][5]],\
                                Cleaned_Measured_Depth_Simulated.Y_3_3[0:No_Detected_Pits.loc[1][7]]], axis=0,\
                                        join='outer', ignore_index=True, keys=None, levels=None, names=None,\
                                        verify_integrity=False, copy=True))

In [ ]:
# Comparing the histograms and finding the bins intervals and frequencies
frequency_info_3 = pd.DataFrame(0, index = range(1, bins_no+1), columns = range(6)).astype(float)
frequency_info_3.columns = ['actual_N','detected_N', 'reported_N', 'estimated_N_WO_POD', 'estimated_N_W_POD',\
                            'rounded_estimated_N_W_POD']

bins_start_3 = np.int(np.min(pd.concat([Initiation_3_Actual_Depth[0],Initiation_2_Actual_Depth[1], Initiation_1_Actual_Depth[2]])))
bins_end_3   = np.int(np.max(pd.concat([Initiation_3_Actual_Depth[0],Initiation_2_Actual_Depth[1], Initiation_1_Actual_Depth[2]])))
(actual_N_3, bins_3, patches) = plt.hist(pd.concat([Initiation_3_Actual_Depth[0],Initiation_2_Actual_Depth[1], Initiation_1_Actual_Depth[2]]), bins_no,\
                                     range=(bins_start_3, bins_end_3),\
                                     histtype='step', normed=False, linewidth=2, color = 'blue',\
                                     label='Distribution of actual samples ILI_3')
(detected_N_3, bins_3, patches3) = plt.hist(Y_3_d[0], bins_no, range=(bins_start_3, bins_end_3),\
                                                 histtype='step', normed=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');


frequency_info_3.detected_N = detected_N_3
frequency_info_3.actual_N = actual_N_3

In [ ]:
# Estimate the number of reported pits given synthetic detected pits
for i in range (1, bins_no+1):
    POFC = np.exp(-(bins_3[i-1]+bins_3[i])/2/credible_pit_depth_threshold)  
    frequency_info_3.reported_N[i] = frequency_info_3.detected_N[i]/(1-POFC)

In [ ]:
# estimate the actual depth of reported pits and find the distribution of R_3
np.random.seed(seed_number)
R_3 = pd.DataFrame(0, index = range(0, np.int(np.sum(frequency_info_3.reported_N))), columns = range(1)).astype(float) 
for i in range(0, np.int(m_3)):
    R_3[0][i] = Y_3_d[0][i]
jj = 0
for i in range(1, bins_no+1):      
    if frequency_info_3.reported_N[i] > frequency_info_3.detected_N[i]:        
        for j in range(1, np.int(frequency_info_3.reported_N[i] - frequency_info_3.detected_N[i])+1):          
            R = np.random.uniform(bins_3[i-1], bins_3[i])
            R_3[0][np.int(m_3) + jj] = R
            jj = jj + 1     
R_3.to_csv('R_3.csv')

In [ ]:
# Fliter true calls measurements from reported measurements
np.random.seed(seed_number)
No_TrueCalled_Pits[2][1] = 0
for item in R_3[0]:           
    if np.random.uniform(0,1) > np.exp(-item/credible_pit_depth_threshold):
        Cleaned_Measured_Depth.Y_3[No_TrueCalled_Pits[2][1]] = item
        No_TrueCalled_Pits[2][1] = No_TrueCalled_Pits[2][1] + 1
print(No_TrueCalled_Pits[2][1])

In [ ]:
# Comparison
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)


ddd = pd.DataFrame(pd.concat([Initiation_1_Actual_Depth[2],\
                              Initiation_2_Actual_Depth[1],\
                              Initiation_3_Actual_Depth[0]],axis=0,\
                            join='outer', ignore_index=True, keys=None, levels=None, names=None,\
                             verify_integrity=False, copy=True))
plt.hist(ddd[0], bins_no, color = 'green', range=(bins_start_3, bins_end_3),\
          histtype='step', density=True, linewidth=2, label='Actual depth at 1_1');
plt.hist(Y_3_d[0], bins_no, color = 'red', range=(bins_start_3, bins_end_3), histtype='step', density=True, linewidth=2,\
         label='Detected');  
plt.hist(R_3[0], bins_no, color = 'blue', range=(bins_start_3, bins_end_3), histtype='step', density=True, linewidth=2,\
         label='Reported');  
plt.hist(Cleaned_Measured_Depth.Y_3[0:No_TrueCalled_Pits[2][1]], bins_no, color = 'yellow', range=(bins_start_3, bins_end_3),\
         histtype='step', density=True, linewidth=2,\
         label='TrueCalls'); 
plt.xlim(bins_start_3, bins_end_3);

plt.title('Updating Hyper-parameters' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
Cleaned_Measured_Depth.to_csv('Cleaned_Measured_Depth.csv')

In [ ]:
No_TrueCalled_Pits

In [ ]:
No_TrueCalled_Pits[2][1]

# 3- Using ILI 1

# 3-1- Estimated D_1 , k , nu and Scale Par in Openbugs

In [ ]:
# Use Cleaned_Measured_Depth.Y_1_1 as the first set of ILI data to estimate the hyper-parameters

In [ ]:
alpha_1_1_actual = k*(ILI_info.ILI_time[0]-0)**nu
alpha_1_1_actual

In [ ]:
# Estimated parameters at ILI_1 by OpenBUGS using true calls data at ILI_1
k_1  = 0.4836
nu_1 = 0.3896
ScalePar_1 = 3.054
alpha_1_1_OpenBugs = 1.504
alpha_1_1_Est = k_1*(ILI_info.ILI_time[0]-0)**nu_1
alpha_1_1_Est

In [ ]:
# Estimated actual depth by OpenBUGS for detected pits at ILI 
D_1_1 = pd.read_excel('D.xlsx', sheetname='D_1_1')

# 3-2- Estimate the number of actual pits and their actual depth at  t_1

In [ ]:
# Comparing the number of pits at each bin of the PMF of the actual depth and the estimated depth without POD


(estimated_N_WO_POD_1, bins_1, patches1) = plt.hist(D_1_1['depth'], bins_no, range=(bins_start_1, bins_end_1),\
                                                 histtype='step', density=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');


In [ ]:

# Calculating number of pits at each bin of the PMF of the estimated depth without POD
(estimated_N_WO_POD_1, bins_1, patches1) = plt.hist(D_1_1['depth'], bins_no, \
                                                 histtype='step', density=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');
frequency_info_1.estimated_N_WO_POD = estimated_N_WO_POD_1

In [ ]:
# Estimate the number of actual pits (M_1_1) given the number of truly detected pits (m_1) at ILI_1:
for i in range (1, bins_no+1):
    POD = 1-np.exp(-(bins_1[i-1]+bins_1[i])/2/detection_threshold)
    frequency_info_1.estimated_N_W_POD[i] = frequency_info_1.estimated_N_WO_POD[i]/POD
frequency_info_1.rounded_estimated_N_W_POD = np.round(frequency_info_1.estimated_N_W_POD, 0)

In [ ]:
frequency_info_1.head()

In [ ]:

p = figure(title = 'Comparison between actual and estimated number of pits at each bin', x_axis_label='actual_N',\
           y_axis_label='rounded_estimated_N_W_POD', plot_height = 700, plot_width = 700)   
p.scatter(actual_N_1, frequency_info_1.rounded_estimated_N_W_POD, line_width = 2, color = 'blue')
p.line([0, np.max(actual_N_1)], [0, np.max(actual_N_1)], line_width = 2, legend ='1 X 1 line', color = 'red')
p.legend.location = "top_left"
show(p) 

In [ ]:
m_1_1 = np.int(np.sum(frequency_info_1.estimated_N_WO_POD))
m_1_1

In [ ]:
M_1_1= np.int(np.sum(frequency_info_1.rounded_estimated_N_W_POD))
M_1_1

In [ ]:
# estimate the actual depth of undetected pits and find the distribution of X_1
np.random.seed(seed_number)
X_1 = pd.DataFrame(0, index = range(0, M_1_1), columns = range(1)).astype(float) 
for i in range(0, m_1_1):
    X_1[0][i] = D_1_1['depth'][i]

jj = 0
for i in range(1, bins_no+1):      
    if frequency_info_1.rounded_estimated_N_W_POD[i] > frequency_info_1.estimated_N_WO_POD[i]:        
        for j in range(1, np.int(frequency_info_1.rounded_estimated_N_W_POD[i] - frequency_info_1.estimated_N_WO_POD[i])+1):          
            X = np.random.uniform(bins_1[i-1], bins_1[i])
            X_1[0][m_1_1 + jj] = X
            jj = jj + 1     
X_1.to_csv('X_1.csv')

In [ ]:
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth(mm)', fontsize = 20)
plt.ylabel('PMF', fontsize = 20)
bins_start_X_1 = np.int(np.min([np.min(Initiation_1_Actual_Depth[0]), np.min(X_1[0])]))
bins_end_X_1   = np.int(np.max([np.max(Initiation_1_Actual_Depth[0]), np.max(X_1[0])]))
plt.hist(Initiation_1_Actual_Depth[0], bins_no, color = 'green', range=(bins_start_X_1, bins_end_X_1),\
         histtype='step', density=True, linewidth=2, label='Simulated actual depth (X_1) at t_1');
plt.hist(X_1[0], bins_no, color = 'red', range=(bins_start_X_1, bins_end_X_1), histtype='step', density=True, linewidth=1,\
         label='Estimated actual depth (X_1) at t_1')  
plt.xlim(bins_start_X_1, 30)
plt.title('' , fontsize = 16)
plt.legend(loc='upper right')

In [ ]:
(actual_N_1, bins_1, patches) = plt.hist(Initiation_1_Actual_Depth[0], bins_no,\
                                     range=(bins_start_X_1, bins_end_X_1),\
                                     histtype='step', density=False, linewidth=2, color = 'blue',\
                                     label='Distribution of actual samples ILI_1_1');
(estimated_N_W_POD_1, bins_1, patches1) = plt.hist(X_1[0], bins_no, range=(bins_start_X_1, bins_end_X_1),\
                                                 histtype='step', density=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');

In [ ]:
def KL_divergences(h1_bin_freq, h2_bin_freq, bins):
    D_KL = 0
    for i in range(0, bins):
        if h2_bin_freq[i] > 0 and h1_bin_freq[i] > 0:
            D_KL += h1_bin_freq[i]*np.log(h1_bin_freq[i]/h2_bin_freq[i])
    return D_KL

In [ ]:
N_Actual_1 = pd.concat([Initiation_1_Actual_Depth[0]]).shape[0]
N_Estimated_1 = X_1.shape[0] 
KLD1 = KL_divergences(actual_N_1/N_Actual_1, estimated_N_WO_POD_1/N_Estimated_1, bins_no)
KLD1

In [ ]:
SKLD1 = (KL_divergences(actual_N_1/N_Actual_1, estimated_N_WO_POD_1/N_Estimated_1, bins_no) + \
         KL_divergences(estimated_N_WO_POD_1/N_Estimated_1, actual_N_1/N_Actual_1, bins_no))/2
SKLD1

In [ ]:
ChiSq_1 = 0
for i in range(0, bins_no):
    if actual_N_1[i] > 0 or estimated_N_W_POD_1[i] > 0:
        ChiSq_1 = ChiSq_1 + (actual_N_1[i]/N_Actual_1 - estimated_N_W_POD_1[i]/N_Estimated_1)**2/\
        (actual_N_1[i]/N_Actual_1 + estimated_N_W_POD_1[i]/N_Estimated_1)
ChiSq_1

# 4- Using ILI 2 

# 4-1- Cluster the ILI_2 measurement data 

In [ ]:
# Estimated shape parameter at ILI 2 for pits that are initiated before ILI 1
alpha_prime_2_1 = k_1 * (ILI_info.ILI_time[1] - 0) ** nu_1
alpha_prime_2_1

In [ ]:
alpha_2_1_actual = k*(ILI_info.ILI_time[1]-0)**nu
alpha_2_1_actual

In [ ]:
alpha_2_2_actual = k*(ILI_info.ILI_time[1]-ILI_info.ILI_time[0])**nu
alpha_2_2_actual

In [ ]:
ScalePar

In [ ]:
# Estimated shape parameter at ILI 2 for pits that are initiated between ILI 1 and 2
alpha_prime_2_2 = k_1 * (ILI_info.ILI_time[1] - ILI_info.ILI_time[0]) ** nu_1
alpha_prime_2_2

In [ ]:
detection_threshold

In [ ]:
𝑀_1_1

In [ ]:
ScalePar_1

In [ ]:
# Using Wolfram Alpha to calculate m_prime_2_1
# WolframAlpha equation: 
# integrate 1870/gamma(1.932)/3.054^1.932*x^(1.932-1)*exp(-x/3.054)*(1-exp(-x/0.84))dx from x=0 to 20
m_prime_2_1 = 1755 # = 𝑀_1_1 ∫𝑔′_2_1(x)𝑃𝑂𝐷(𝑥)𝑑(𝑥)
m_prime_2_2 = m_2 - m_prime_2_1
m_prime_2_2

In [ ]:
np.random.seed(seed_number)
# Estimate the expected distribution of measuremnets at t_2
# For ILI_2_1
Y_prime_2_1 = pd.DataFrame(np.zeros(1))
D_prime_2_1 = np.random.gamma(alpha_prime_2_1, scale = ScalePar_1, size=m_prime_2_1) 

j = 0
for i in range (0, m_prime_2_1):
    Y_prime_2_1.loc[j] = a_ILI + b_ILI * D_prime_2_1[i] + np.random.normal (0, ILI_error_STD)  
    j = j + 1
mean_Y_prime_2_1 = np.mean(Y_prime_2_1)
STD_Y_prime_2_1  = np.std(Y_prime_2_1)

# For ILI_2_2
Y_prime_2_2 = pd.DataFrame(np.zeros(1))
D_prime_2_2 = np.random.gamma(alpha_prime_2_2, scale=ScalePar_1, size=m_prime_2_2[1]) 

j = 0
for i in range (0, m_prime_2_2[1]):
    Y_prime_2_2.loc[j] = a_ILI + b_ILI * D_prime_2_2[i] + np.random.normal (0, ILI_error_STD)  
    j = j + 1
mean_Y_prime_2_2 = np.mean(Y_prime_2_2)
STD_Y_prime_2_2 = np.std(Y_prime_2_2)

In [ ]:
# Visualization of expected measured data at ILI_2
tt = np.linspace(-10, 50, 1000)
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)
plt.plot(tt, norm.pdf(tt, loc = mean_Y_prime_2_1, scale = STD_Y_prime_2_1), c='green', linestyle='--', alpha=0.5,  linewidth=3,\
         label='y_prime_ILI_2_1') 

plt.plot(tt, norm.pdf(tt, loc = mean_Y_prime_2_2, scale =  STD_Y_prime_2_2), c='Orange', linestyle='--', alpha=0.5,  linewidth=3,\
          label='y_prime_ILI_2_2')

plt.xlim([-10, 20]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
# Combine detected measurements at those pits that are detected at t_2 and initiated at 1st and 2nd intervals
Y_2_with_zeros = pd.DataFrame(Cleaned_Measured_Depth.Y_2)

In [ ]:
# Remove zeros from Y_2_with_zeros and save them in an array
Y_2_array = pd.DataFrame(np.zeros(1))
j = 0
for i in range (0, Cleaned_Measured_Depth.Y_2.shape[0]):       
    if Y_2_with_zeros.Y_2[i] > 0 or Y_2_with_zeros.Y_2[i] < 0:
        Y_2_array.loc[j] = Y_2_with_zeros.Y_2[i] 
        j = j + 1


In [ ]:
# Figureing out which detected pit at t_2 belongs to which population 2_1 or 2_2
Y_2 = pd.DataFrame(0, index = range(0, len(Y_2_array)), columns = range(6)).astype(float) 
Y_2.columns = ['measurement', 'Pr_yi_given_2_1', 'Pr_yi_given_2_2','Pr_2_1_given_yi', 'Pr_2_2_given_yi',\
               'cluster_index']
Y_2.measurement = Y_2_array

In [ ]:
# Finding the start and end points of the bins of Y_2 which will be the same for Y_2_1 and Y_2_2 as well
measurement_bins_start_2 = np.int(np.min(Y_2.measurement))
measurement_bins_end_2   = np.int(np.max(Y_2.measurement))
(measurement_N_2, measurement_bins_2, patches) = plt.hist(Y_2.measurement, bins_no,\
                                     range=(measurement_bins_start_2, measurement_bins_end_2),\
                                     histtype='step', density=False, linewidth=2, color = 'blue',\
                                     label='');


In [ ]:
# Finding the frequency of each bin for Y_prime_2_1 using the same start and end bin as Y_2
(Y_prime_2_1_N, measurement_bins_2, patches) = plt.hist(Y_prime_2_1[0], bins_no,\
                                     range=(measurement_bins_start_2, measurement_bins_end_2),\
                                     histtype='step', density=False, linewidth=2, color = 'blue',\
                                     label='Y_prime_2_1');


In [ ]:
# Finding the frequency of each bin for Y_prime_2_1 using the same start and end bin as Y_2
(Y_prime_2_2_N, measurement_bins_2, patches) = plt.hist(Y_prime_2_2[0], bins_no,\
                                     range=(measurement_bins_start_2, measurement_bins_end_2),\
                                     histtype='step', density=False, linewidth=2, color = 'red',\
                                     label='Y_prime_2_2');


In [ ]:
# Figureing out which detected pit at t_2 belongs to which population 2_1 or 2_2 
Y_2_1_bins_freq = pd.DataFrame(0, index = range(0, bins_no), columns = range(1)).astype(int) 
Y_2_2_bins_freq = pd.DataFrame(0, index = range(0, bins_no), columns = range(1)).astype(int) 

expected_bin_no_2_1 = pd.DataFrame(0, index = range(0, bins_no), columns = range(1)).astype(int) 
expected_bin_no_2_2 = pd.DataFrame(0, index = range(0, bins_no), columns = range(1)).astype(int) 

bin_index_2 = pd.DataFrame(np.zeros(len(Y_2_array))).astype(int)

Y_prime_2_1_No = 0
Y_prime_2_2_No = 0


# identifying the bin index for each measured value
for i in range(0, len(Y_2_array)):    
    for j in range(0, bins_no):
        if Y_2.measurement[i] > measurement_bins_2[j] and Y_2.measurement[i] < measurement_bins_2[j+1]:
            bin_index_2.iloc[i] = np.int(j)
            break    

for i in range(0, len(Y_2_array)):     
    if Y_2_1_bins_freq[0][bin_index_2[0][i]] < Y_prime_2_1_N[bin_index_2[0][i]]:            
            Y_2.cluster_index[i] = 1
            Y_2_1_bins_freq[0][bin_index_2[0][i]] = Y_2_1_bins_freq[0][bin_index_2[0][i]] + 1
            Y_prime_2_1_No = Y_prime_2_1_No + 1
    else:
        Y_2.cluster_index[i] = 2
        Y_2_2_bins_freq[0][bin_index_2[0][i]] = Y_2_2_bins_freq[0][bin_index_2[0][i]] + 1
        Y_prime_2_2_No = Y_prime_2_2_No + 1     

In [ ]:
# Find the arrays of Y_2_1 and Y_2_2
Y_2_1 = pd.DataFrame(0, index = range(1, Y_prime_2_1_No+1), columns = range(1)).astype(float)
Y_2_2 = pd.DataFrame(0, index = range(1, Y_prime_2_2_No+1), columns = range(1)).astype(float)
j1 = 1
j2 = 1
for i in range (0, len(Y_2_array)): 
    if Y_2.cluster_index[i] == 1:
        Y_2_1[0][j1] = Y_2.measurement[i]  
        j1 = j1 + 1
    elif Y_2.cluster_index[i] == 2:
        Y_2_2[0][j2] = Y_2.measurement[i]  
        j2 = j2 + 1    

In [ ]:
Y_2.to_csv('Y_2.csv')

In [ ]:
Y_2_1.to_csv('Y_2_1.csv')
Y_2_2.to_csv('Y_2_2.csv')

# 4-2- Use the Clustered ILI_2 measurement data to update the hyper-parameters and estimate the actual depth of the detected pits in OpenBUGS

In [ ]:
# Actual shape parameter at ILI 2 for pits that are initiated before ILI 1
alpha_2_1_actual = k * (ILI_info.ILI_time[1] - 0) ** nu
alpha_2_1_actual

In [ ]:
# Actual shape parameter at ILI 2 for pits that are initiated between ILI 1 and ILI 2
alpha_2_2_actual = k * (ILI_info.ILI_time[1] - ILI_info.ILI_time[0]) ** nu
alpha_2_2_actual

In [ ]:
k

In [ ]:
nu

In [ ]:
# Estimated parameters at ILI 2 by OpenBUGS at t_2
k_2  = 0.5919
nu_2 = 0.3115
ScalePar_2 = 2.983
alpha_2_1_Est = 1.787
alpha_2_2_Est = 0.9753

In [ ]:
# Estimated actual depth by OpenBUGS for detected pits at ILI 
#D_2_1 = pd.read_excel('D.xlsx', sheetname='D_2_1')
#D_2_2 = pd.read_excel('D.xlsx', sheetname='D_2_2')
D_2 = pd.read_excel('D.xlsx', sheetname='D_2')

In [ ]:
p = figure(title = 'Comparison between actual and estimated number of pits at each bin', x_axis_label='actual_N',\
           y_axis_label='rounded_estimated_N_W_POD', plot_height = 700, plot_width = 700)   
p.line( tt, k*(tt-t0)**nu*ScalePar, line_width = 2, legend ='actual',  color = 'red')
p.line( tt, k_1*(tt-t0)**nu_1*ScalePar_1, line_width = 2, legend ='Estimated 1', color = 'blue')
p.line( tt, k_2*(tt-t0)**nu_2*ScalePar_2, line_width = 2, legend ='Estimated 2', color = 'green')

p.legend.location = "top_left"

show(p) 

# 4-3- Estimate the number of actual pits and their actual depth at  t_2

In [ ]:
# Calculating number of pits at each bin of the PMF of actual depth and estimated depth without POD
(estimated_N_WO_POD_2, bins_2, patches2) = plt.hist(D_2['depth'], bins_no, \
                                                 histtype='step', density=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');
frequency_info_2.estimated_N_WO_POD = estimated_N_WO_POD_2

In [ ]:
# Estimate the number of actual pits (M_2) given the number of truly detected pits (m_2) at ILI_2:
for i in range (1, bins_no+1):
    POD = 1-np.exp(-(bins_2[i-1]+bins_2[i])/2/detection_threshold)
    frequency_info_2.estimated_N_W_POD[i] = frequency_info_2.estimated_N_WO_POD[i]/POD
frequency_info_2.rounded_estimated_N_W_POD = np.round(frequency_info_2.estimated_N_W_POD, 0)

In [ ]:
frequency_info_2.head()

In [ ]:
m_2

In [ ]:
# Estimated number of existing pits at t_2
M_2 = np.int(np.sum(frequency_info_2.rounded_estimated_N_W_POD))
M_2

In [ ]:
M_2-No_TrueCalled_Pits[1][1]

In [ ]:
np.random.seed(seed_number)
# Find the PMF of X_2
X_2 = pd.DataFrame(0, index = range(0, M_2), columns = range(1)).astype(float) 
for i in range(0, No_TrueCalled_Pits[1][1]): # change this to m_2
    X_2[0][i] = D_2['depth'][i]
# estimate the actual depth of undetected pits and find the distribution of X_2

jj = 0
for i in range(1, bins_no+1):      
    if frequency_info_2.rounded_estimated_N_W_POD[i] > frequency_info_2.estimated_N_WO_POD[i]:        
        for j in range(1, np.int(frequency_info_2.rounded_estimated_N_W_POD[i] - frequency_info_2.estimated_N_WO_POD[i])+1):          
            X = np.random.uniform(bins_2[i-1], bins_2[i])
            X_2[0][No_TrueCalled_Pits[1][1] + jj] = X # change this to m_2
            jj = jj + 1     
X_2.to_csv('X_2.csv')
print(jj)

In [ ]:
# Visual comparison between actual and estimated depth for X_2
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth (mm)', fontsize = 20)
plt.ylabel('PMF', fontsize = 20)
bins_start_X_2 = np.int(np.min([np.min(pd.concat([Initiation_2_Actual_Depth[0], Initiation_1_Actual_Depth[1]])),\
                              np.min(X_2[0])]))
bins_end_X_2   = np.int(np.max([np.max(pd.concat([Initiation_2_Actual_Depth[0], Initiation_1_Actual_Depth[1]])),\
                              np.max(X_2[0])]))
plt.hist(pd.concat([Initiation_2_Actual_Depth[0], Initiation_1_Actual_Depth[1]]), bins_no, color = 'green',\
         range=(bins_start_X_2, bins_end_X_2), histtype='step', density=True, linewidth=2,\
         label='Simulated actual depth (X_2) at t_2');
plt.hist(X_2[0], bins_no, color = 'red', range=(bins_start_X_2, bins_end_X_2), histtype='step', density=True,\
         linewidth=1, label='Estimated actual depth (X_2) at t_2');

plt.xlim([bins_start_X_2, 30]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
(actual_N_2, bins_2, patches) = plt.hist(pd.concat([Initiation_2_Actual_Depth[0], Initiation_1_Actual_Depth[1]]),\
                                         bins_no, range=(bins_start_X_2, bins_end_X_2),\
                                     histtype='step', density=False, linewidth=2, color = 'blue',\
                                     label='Distribution of actual samples ILI_2');
(estimated_N_W_POD_2, bins_2, patches2) = plt.hist(X_2[0], bins_no, range=(bins_start_X_2, bins_end_X_2),\
                                                 histtype='step', density=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');

In [ ]:
N_Actual_2 = pd.concat([Initiation_2_Actual_Depth[0], Initiation_1_Actual_Depth[1]]).shape[0]
N_Estimated_2 = X_2.shape[0] 
KLD2 = KL_divergences(actual_N_2/N_Actual_2, estimated_N_WO_POD_2/N_Estimated_2, bins_no)
KLD2

In [ ]:
SKLD2 = (KL_divergences(actual_N_2/N_Actual_2, estimated_N_WO_POD_2/N_Estimated_2, bins_no) + \
         KL_divergences(estimated_N_WO_POD_2/N_Estimated_2, actual_N_2/N_Actual_2, bins_no))/2
SKLD2

In [ ]:
ChiSq_2 = 0
for i in range(0, bins_no):
    if actual_N_2[i] > 0 or estimated_N_W_POD_2[i] > 0:
        ChiSq_2 = ChiSq_2 + (actual_N_2[i]/N_Actual_2 - estimated_N_W_POD_2[i]/N_Estimated_2)**2/\
        (actual_N_2[i]/N_Actual_2 + estimated_N_W_POD_2[i]/N_Estimated_2)
ChiSq_2

# 5- Using ILI 3 data

# 5-1- Cluster the ILI_3 measurement data 

In [ ]:
# Estimated shape parameter at ILI 3 for pits that are initiated before ILI 1
alpha_prime_3_1 = k_2 * (ILI_info.ILI_time[2] - 0) ** nu_2
alpha_prime_3_1

In [ ]:
alpha_3_1_actual = k * (ILI_info.ILI_time[2] - 0) ** nu
alpha_3_1_actual

In [ ]:
# Estimated shape parameter at ILI 3 for pits that are initiated between ILI 1 and 2
alpha_prime_3_2 = k_2 * (ILI_info.ILI_time[2] - ILI_info.ILI_time[0]) ** nu_2
alpha_prime_3_2

In [ ]:
alpha_3_2_actual = k * (ILI_info.ILI_time[2] - ILI_info.ILI_time[0]) ** nu
alpha_3_2_actual

In [ ]:
# Estimated shape parameter at ILI 3 for pits that are initiated between ILI 2 and 3
alpha_prime_3_3 = k_2 * (ILI_info.ILI_time[2] - ILI_info.ILI_time[1]) ** nu_2
alpha_prime_3_3

In [ ]:
alpha_3_3_actual = k * (ILI_info.ILI_time[2] - ILI_info.ILI_time[1]) ** nu
alpha_3_3_actual

In [ ]:
𝑀_1_1

In [ ]:
M_2

In [ ]:
M_2-𝑀_1_1

In [ ]:
m_3

In [ ]:
ScalePar

In [ ]:
ScalePar_2

In [ ]:
# Using Wolfram Alpha to calculate m_prime_3_3
# = 𝑀_1_1 ∫𝑔′_3_1(x)𝑃𝑂𝐷(𝑥)𝑑(𝑥), 𝑔′_3_1(x) = gamma(alpha_prime_3_1, ScalePar_1) = 
# integrate 1870/gamma(1.8676)/2.983^1.8676*x^(1.8676-1)*exp(-x/2.983)*(1-exp(-x/0.84))dx from x=0 to 20
m_prime_3_1 = 1745 
# integrate 464/gamma(0.779)/1.765^0.779*x^(0.779-1)*exp(-x/1.765)*(1-exp(-x/0.84))dx from x=0 to 20
m_prime_3_2 = 346 # = (M_2-𝑀_1_1) ∫𝑔′_3_2(x)𝑃𝑂𝐷(𝑥)𝑑(𝑥), 𝑔′_3_2(x) = gamma(alpha_prime_3_2, ScalePar_2)
m_prime_3_3 = m_3 - m_prime_3_1 - m_prime_3_2
m_prime_3_3

In [ ]:
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)
plt.plot(tt, gamma.pdf(tt, alpha_3_1_actual, scale = ScalePar), c='green', linestyle='--', alpha=0.5,  linewidth=3,\
         label='Actual pit depth dist.') 

plt.plot(tt, gamma.pdf(tt, alpha_prime_3_1, scale = ScalePar_1), c='Orange', linestyle='--', alpha=0.5,  linewidth=3,\
          label='Estimated pit depth dist.')

plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
Y_3_with_zeros = pd.DataFrame(Cleaned_Measured_Depth.Y_3)

In [ ]:
# Remove zeros from Y_3_with_zeros
Y_3_array = pd.DataFrame(np.zeros(1)).astype(float)
j = 0
for i in range (0, Cleaned_Measured_Depth.Y_3.shape[0]):       
    if Y_3_with_zeros.Y_3[i] > 0 or Y_3_with_zeros.Y_3[i] < 0:
        Y_3_array.loc[j] = Y_3_with_zeros.Y_3[i]  
        j = j + 1

In [ ]:
# Store the ILI 3 cleaned data in one new file Y_3 for clustering
Y_3 = pd.DataFrame(0, index = range(0, len(Y_3_array)), columns = range(8)).astype(float) 
Y_3.columns = ['measurement', 'Pr_yi_given_3_1', 'Pr_yi_given_3_2','Pr_yi_given_3_3','Pr_3_1_given_yi',\
               'Pr_3_2_given_yi', 'Pr_3_3_given_yi', 'cluster_index']
Y_3.measurement = Y_3_array

In [ ]:
# Finding the start and end points of the bins of Y_3 which will be the same for Y_3_1, Y_3_2, and Y_3_3 as well
measurement_bins_start_3 = np.int(np.min(Y_3.measurement))
measurement_bins_end_3   = np.int(np.max(Y_3.measurement))
(measurement_N_3, measurement_bins_3, patches) = plt.hist(Y_3.measurement, bins_no,\
                                     range=(measurement_bins_start_3, measurement_bins_end_3),\
                                     histtype='step', density=False, linewidth=2, color = 'blue',\
                                     label='Y_3');


In [ ]:
np.random.seed(seed_number)
# Estimate the expected distribution of measuremnets at t_3
# For ILI_3_1
Y_prime_3_1 = pd.DataFrame(np.zeros(1))
D_prime_3_1 = np.random.gamma(alpha_prime_3_1, scale = ScalePar_2, size = m_prime_3_1) 

j = 0
for i in range (0, np.int(m_prime_3_1)):
    Y_prime_3_1.loc[j] = a_ILI + b_ILI * D_prime_3_1[i] + np.random.normal (0, ILI_error_STD)  
    j = j + 1
mean_Y_prime_3_1 = np.mean(Y_prime_3_1)
STD_Y_prime_3_1 = np.std(Y_prime_3_1)

# For ILI_3_2
Y_prime_3_2 = pd.DataFrame(np.zeros(1))
D_prime_3_2 = np.random.gamma(alpha_prime_3_2, scale = ScalePar_2, size = m_prime_3_2) 

j = 0
for i in range (0, np.int(m_prime_3_2)):
    Y_prime_3_2.loc[j] = a_ILI + b_ILI * D_prime_3_2[i] + np.random.normal (0, ILI_error_STD)  
    j = j + 1
mean_Y_prime_3_2 = np.mean(Y_prime_3_2)
STD_Y_prime_3_2 = np.std(Y_prime_3_2)

# For ILI_3_3
Y_prime_3_3 = pd.DataFrame(np.zeros(1))
D_prime_3_3 = np.random.gamma(alpha_prime_3_3, scale = ScalePar_2, size = m_prime_3_3[1]) 

j = 0
for i in range (0, np.int(m_prime_3_3)):
    Y_prime_3_3.loc[j] = a_ILI + b_ILI * D_prime_3_3[i] + np.random.normal (0, ILI_error_STD)  
    j = j + 1
mean_Y_prime_3_3 = np.mean(Y_prime_3_3)
STD_Y_prime_3_3 = np.std(Y_prime_3_3)

In [ ]:
# Visual comparison between actual and clustered measurements
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)

plt.hist(Y_3_array[0], bins_no, color = 'black', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step',\
         density=False, linewidth=2,label='Y_3_array');

# plt.hist(Cleaned_Measured_Depth.Y_3_1[1:1443], bins_no, color = 'blue', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', normed=False, linewidth=2,\
#          label='Cleaned_Measured_Depth.Y_3_1');
# plt.hist(Cleaned_Measured_Depth.Y_3_2[1:269], bins_no, color = 'red', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', normed=False, linewidth=2,\
#          label='Cleaned_Measured_Depth.Y_3_2');
# plt.hist(Cleaned_Measured_Depth.Y_3_3[1:109], bins_no, color = 'green', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', normed=False, linewidth=2,\
#          label='Cleaned_Measured_Depth.Y_3_3');

plt.hist(Y_prime_3_1[0], bins_no, color = 'blue', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', density=False, linewidth=1,\
         label='Expected_Measured_Depth, Y_prime_3_1');
plt.hist(Y_prime_3_2[0], bins_no, color = 'red', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', density=False, linewidth=1,\
         label='Expected_Measured_Depth, Y_prime_3_2');
plt.hist(Y_prime_3_3[0], bins_no, color = 'green', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', density=False, linewidth=1,\
         label='Expected_Measured_Depth, Y_prime_3_3');

plt.xlim([measurement_bins_start_3, 20]);
plt.title('' , fontsize = measurement_bins_end_3)
plt.legend(loc='upper right');

In [ ]:
# Finding the frequency of each bin for Y_prime_3_1 using the same start and end bin as Y_3
(Y_prime_3_1_N, measurement_bins_3, patches) = plt.hist(Y_prime_3_1[0], bins_no,\
                                     range=(measurement_bins_start_3, measurement_bins_end_3),\
                                     histtype='step', density=False, linewidth=2, color = 'blue',\
                                     label='Y_prime_3_1');


In [ ]:
# Finding the frequency of each bin for Y_prime_3_2 using the same start and end bin as Y_3
(Y_prime_3_2_N, measurement_bins_3, patches) = plt.hist(Y_prime_3_2[0], bins_no,\
                                     range=(measurement_bins_start_3, measurement_bins_end_3),\
                                     histtype='step', density=False, linewidth=2, color = 'red',\
                                     label='Y_prime_3_2');


In [ ]:
# Finding the frequency of each bin for Y_prime_3_3 using the same start and end bin as Y_3
(Y_prime_3_3_N, measurement_bins_3, patches) = plt.hist(Y_prime_3_3[0], bins_no,\
                                     range=(measurement_bins_start_3, measurement_bins_end_3),\
                                     histtype='step', density=False, linewidth=2, color = 'green',\
                                     label='Y_prime_3_3');


In [ ]:
# Figureing out which detected pit at t_3 belongs to which population 3_1 or 3_2 or 3_3
Y_3_1_bins_freq = pd.DataFrame(0, index = range(0, bins_no), columns = range(1)).astype(int) 
Y_3_2_bins_freq = pd.DataFrame(0, index = range(0, bins_no), columns = range(1)).astype(int) 
Y_3_3_bins_freq = pd.DataFrame(0, index = range(0, bins_no), columns = range(1)).astype(int) 

bin_index_3 = pd.DataFrame(np.zeros(len(Y_3_array))).astype(int)

Y_prime_3_1_No = 0
Y_prime_3_2_No = 0
Y_prime_3_3_No = 0

# identifying the bin index for each measured value
for i in range(0, len(Y_3_array)):    
    for j in range(0, bins_no):
        if Y_3.measurement[i] > measurement_bins_3[j] and Y_3.measurement[i] < measurement_bins_3[j+1]:
            bin_index_3.iloc[i] = np.int(j)
            break     
    
for i in range(0, len(Y_3_array)):     
    if Y_3_1_bins_freq[0][bin_index_3[0][i]] < Y_prime_3_1_N[bin_index_3[0][i]]:            
            Y_3.cluster_index[i] = 1
            Y_3_1_bins_freq[0][bin_index_3[0][i]] = Y_3_1_bins_freq[0][bin_index_3[0][i]] + 1
            Y_prime_3_1_No = Y_prime_3_1_No + 1
    elif Y_3_2_bins_freq[0][bin_index_3[0][i]] < Y_prime_3_2_N[bin_index_3[0][i]]:            
        Y_3.cluster_index[i] = 2
        Y_3_2_bins_freq[0][bin_index_3[0][i]] = Y_3_2_bins_freq[0][bin_index_3[0][i]] + 1
        Y_prime_3_2_No = Y_prime_3_2_No + 1 
    else:
        Y_3.cluster_index[i] = 3
        Y_3_3_bins_freq[0][bin_index_3[0][i]] = Y_3_3_bins_freq[0][bin_index_3[0][i]] + 1
        Y_prime_3_3_No = Y_prime_3_3_No + 1 
    

In [ ]:
# Visualization of PDF of expected measured data at ILI_3
tt = np.linspace(-10, 50, 1000)
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)
plt.plot(tt, norm.pdf(tt, mean_Y_prime_3_1, STD_Y_prime_3_1), c='green', linestyle='--', alpha=0.5,  linewidth=3,\
         label='y_prime_ILI_3_1') 
plt.plot(tt, norm.pdf(tt, mean_Y_prime_3_2, STD_Y_prime_3_2), c='orange', linestyle='--', alpha=0.5,  linewidth=3,\
          label='y_prime_ILI_3_2')
plt.plot(tt, norm.pdf(tt, mean_Y_prime_3_3, STD_Y_prime_3_3), c='blue', linestyle='--', alpha=0.5,  linewidth=3,\
          label='y_prime_ILI_3_3')
plt.xlim([-20, 20]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
# Visualization of PMF of expected measured data at ILI_3
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)
# Change these bins start and end to 3
plt.hist(Y_prime_3_1[0], bins_no, color = 'blue', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', density=False, linewidth=2,\
         label='Expected_Measured_Depth, Y_prime_3_1');
plt.hist(Y_prime_3_2[0], bins_no, color = 'red', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', density=False, linewidth=1,\
         label='Expected_Measured_Depth, Y_prime_3_2');
plt.hist(Y_prime_3_3[0], bins_no, color = 'green', range=(measurement_bins_start_3, measurement_bins_end_3), histtype='step', density=False, linewidth=1,\
         label='Expected_Measured_Depth, Y_prime_3_3');

plt.xlim([-2, 20]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
Y_3.to_csv('Y_3.csv')

In [ ]:
# Find the arrays of Y_3_1 and Y_3_2 and Y_3_3
Y_3_1 = pd.DataFrame(0, index = range(1, Y_prime_3_1_No+1), columns = range(1)).astype(float)
Y_3_2 = pd.DataFrame(0, index = range(1, Y_prime_3_2_No+1), columns = range(1)).astype(float)
Y_3_3 = pd.DataFrame(0, index = range(1, Y_prime_3_3_No+1), columns = range(1)).astype(float)
j1 = 1
j2 = 1
j3 = 1
for i in range (0, len(Y_3_array)): 
    if Y_3.cluster_index[i] == 1:
        Y_3_1.loc[j1][0] = Y_3.measurement[i]  
        j1 = j1 + 1
    elif Y_3.cluster_index[i] == 2:
        Y_3_2.loc[j2][0] = Y_3.measurement[i]  
        j2 = j2 + 1
    elif Y_3.cluster_index[i] == 3:
        Y_3_3.loc[j3][0] = Y_3.measurement[i]  
        j3 = j3 + 1

In [ ]:
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth', fontsize = 16)
plt.ylabel('PDF', fontsize = 16)


plt.hist(Y_3.measurement, bins_no, color = 'black', range=(bins_start_2, bins_end_2), histtype='step', density=False, linewidth=2,\
         label='Y_3');
plt.hist(Y_3_1[0], bins_no, color = 'blue', range=(bins_start_2, bins_end_2), histtype='step', density=False, linewidth=1,\
         label='Y_3_1');
plt.hist(Y_3_2[0], bins_no, color = 'red', range=(bins_start_2, bins_end_2), histtype='step', density=False, linewidth=1,\
         label='Y_3_2');
plt.hist(Y_3_3[0], bins_no, color = 'green', range=(bins_start_2, bins_end_2), histtype='step', density=False, linewidth=1,\
         label='Y_3_3');


plt.xlim([-2, 20]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
Y_3_1.to_csv('Y_3_1.csv')
Y_3_2.to_csv('Y_3_2.csv')
Y_3_3.to_csv('Y_3_3.csv')
Y_3.to_csv('Y_2.csv')

### 5-2- Use the Clustered ILI_3 measurement data to update the hyper-parameters and estimate the actual depth of the detected pits in OpenBUGS

In [ ]:
# Actual shape parameter at ILI 3 for pits that are initiated before ILI 1
alpha_actual_3_1 = k * (ILI_info.ILI_time[2] - 0) ** nu
alpha_actual_3_1

In [ ]:
# Actual shape parameter at ILI 3 for pits that are initiated between ILI 1 and ILI 2
alpha_actual_3_2 = k * (ILI_info.ILI_time[2] - ILI_info.ILI_time[0]) ** nu
alpha_actual_3_2

In [ ]:
# Actual shape parameter at ILI 3 for pits that are initiated between ILI 2 and ILI 3
alpha_actual_3_3 = k * (ILI_info.ILI_time[2] - ILI_info.ILI_time[1]) ** nu
alpha_actual_3_3

In [ ]:
# Estimated parameters at ILI 3 by OpenBUGS before considering POD 
k_3  = 0.1604
nu_3 = 0.6631
ScalePar_3 = 3.169
alpha_3_1_Est = 1.852
alpha_3_2_Est = 0.7384
alpha_3_3_Est = 0.466

In [ ]:
# Estimated actual depth by OpenBUGS for detected pits at ILI 3
#D_3_1 = pd.read_excel('D.xlsx', sheetname='D_3_1')
#D_3_2 = pd.read_excel('D.xlsx', sheetname='D_3_2')
#D_3_3 = pd.read_excel('D.xlsx', sheetname='D_3_3')
D_3   = pd.read_excel('D.xlsx', sheetname='D_3')

# 5-3- Estimate the number of actual pits and their actual depth at  t_3

In [ ]:
# Calculating number of pits at each bin of the PMF of actual depth of truly detected pits at t_3
(estimated_N_WO_POD_3, bins_3, patches3) = plt.hist(D_3['depth'], bins_no, \
                                                 histtype='step', density=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');
frequency_info_3.actual_N = actual_N_3
frequency_info_3.estimated_N_WO_POD = estimated_N_WO_POD_3

In [ ]:
# Estimate the number of actual pits (M_3) given the number of truly detected pits (m_3) at ILI_3:
for i in range (1, bins_no+1):
    POD = 1-np.exp(-(bins_3[i-1]+bins_3[i])/2/detection_threshold)
    frequency_info_3.estimated_N_W_POD[i] = frequency_info_3.estimated_N_WO_POD[i]/POD
frequency_info_3.rounded_estimated_N_W_POD = np.round(frequency_info_3.estimated_N_W_POD, 0)

In [ ]:
frequency_info_3.head()

In [ ]:
# Estimated number of existing pits at t_3
M_3 = np.int(np.sum(frequency_info_3.rounded_estimated_N_W_POD))
M_3

In [ ]:
np.random.seed(seed_number)
X_3 = pd.DataFrame(0, index = range(0, M_3), columns = range(1)).astype(float) 
for i in range(0, No_TrueCalled_Pits[2][1]): # change this to m_2
    X_3[0][i] = D_3['depth'][i]
# estimate the actual depth of undetected pits and find the distribution of X_3

jj = 0
for i in range(1, bins_no+1):      
    if frequency_info_3.rounded_estimated_N_W_POD[i] > frequency_info_3.estimated_N_WO_POD[i]:        
        for j in range(1, np.int(frequency_info_3.rounded_estimated_N_W_POD[i] - frequency_info_3.estimated_N_WO_POD[i])+1):          
            X = np.random.uniform(bins_3[i-1], bins_3[i])
            X_3[0][No_TrueCalled_Pits[2][1] + jj] = X # change this to m_3
            jj = jj + 1     
X_3.to_csv('X_3.csv')

In [ ]:
# Visual comparison between actual and estimated depth for X_3
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth (mm)', fontsize = 20)
plt.ylabel('PMF', fontsize = 20)
bins_start_X_3 = np.int(np.min([np.min(pd.concat([Initiation_3_Actual_Depth[0], Initiation_2_Actual_Depth[1],\
                                                Initiation_1_Actual_Depth[2]])), np.min(X_3[0])]))
bins_end_X_3   = np.int(np.max([np.max(pd.concat([Initiation_3_Actual_Depth[0], Initiation_2_Actual_Depth[1],\
                                                Initiation_1_Actual_Depth[2]])), np.max(X_3[0])]))
plt.hist(pd.concat([Initiation_3_Actual_Depth[0], Initiation_2_Actual_Depth[1], Initiation_1_Actual_Depth[2]]),\
         bins_no, color = 'green', range=(bins_start_X_3, bins_end_X_3), histtype='step', density=True, linewidth=2,\
         label='Simulated actual depth (X_3) at t_3');
plt.hist(X_3[0], bins_no, color = 'red', range=(bins_start_X_3, bins_end_X_3), histtype='step', density=True, linewidth=1,\
         label='Estimated actual depth (X_3) at t_3');

plt.xlim([bins_start_X_3, 30]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
(actual_N_3, bins_3, patches) = plt.hist(pd.concat([Initiation_3_Actual_Depth[0], Initiation_2_Actual_Depth[1],\
                                                    Initiation_1_Actual_Depth[2]]), bins_no, \
                                         range=(bins_start_X_3, bins_end_X_3), histtype='step', density=False, linewidth=2,\
                                         color = 'blue', label='Distribution of actual samples ILI_3');
(estimated_N_W_POD_3, bins_3, patches3) = plt.hist(X_3[0], bins_no, range=(bins_start_X_3, bins_end_X_3),\
                                                 histtype='step', density=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');


In [ ]:
N_Actual_3 = pd.concat([Initiation_3_Actual_Depth[0],\
                Initiation_2_Actual_Depth[1], Initiation_1_Actual_Depth[2]]).shape[0]
N_Estimated_3 = X_3.shape[0] 
KLD3 = KL_divergences(actual_N_3/N_Actual_3, estimated_N_WO_POD_3/N_Estimated_3, bins_no)
KLD3

In [ ]:
SKLD3 = (KL_divergences(actual_N_3/N_Actual_3, estimated_N_WO_POD_3/N_Estimated_3, bins_no) + \
         KL_divergences(estimated_N_WO_POD_3/N_Estimated_3, actual_N_3/N_Actual_3, bins_no))/2
SKLD3

In [ ]:
ChiSq_3 = 0
for i in range(0, bins_no):
    if actual_N_3[i] > 0 or estimated_N_W_POD_3[i] > 0:
        ChiSq_3 = ChiSq_3 + (actual_N_3[i]/N_Actual_3 - estimated_N_W_POD_3[i]/N_Estimated_3)**2/\
        (actual_N_3[i]/N_Actual_3 + estimated_N_W_POD_3[i]/N_Estimated_3)
ChiSq_3

# 6- Prediction step at time t_4

In [ ]:
HPP_rate = (M_1_1+M_2 - M_1_1+M_3 - M_2)/(ILI_info.ILI_time[2])
HPP_rate

In [ ]:
M_4_4 = np.int((ILI_info.ILI_time[3]-ILI_info.ILI_time[2])*HPP_rate) # this is the mean value of numbers
M_4_4

In [ ]:
alpha_actual_4_1 = k_3 * (ILI_info.ILI_time[3] - 0) ** nu_3
alpha_actual_4_2 = k_3 * (ILI_info.ILI_time[3] - ILI_info.ILI_time[0]) ** nu_3
alpha_actual_4_3 = k_3 * (ILI_info.ILI_time[3] - ILI_info.ILI_time[1]) ** nu_3
alpha_actual_4_4 = k_3 * (ILI_info.ILI_time[3] - ILI_info.ILI_time[2]) ** nu_3


In [ ]:
np.random.seed(seed_number)
X_4_1 = pd.DataFrame(0, index = range(1, M_1_1+1), columns = range(1)).astype(float)
X_4_1[0] = np.random.gamma(alpha_actual_4_1, scale=ScalePar_3, size=M_1_1) 
X_4_2 = pd.DataFrame(0, index = range(1, M_2 - M_1_1+1), columns = range(1)).astype(float)
X_4_2[0] = np.random.gamma(alpha_actual_4_2, scale=ScalePar_3, size=M_2 - M_1_1)
X_4_3 = pd.DataFrame(0, index = range(1, M_3 - M_2+1), columns = range(1)).astype(float)
X_4_3[0] = np.random.gamma(alpha_actual_4_3, scale=ScalePar_3, size= M_3 - M_2) 
X_4_4 = pd.DataFrame(0, index = range(1, M_4_4+1), columns = range(1)).astype(float)
X_4_4[0] = np.random.gamma(alpha_actual_4_4, scale=ScalePar_3, size=M_4_4) 
X_4 = pd.concat([X_4_1, X_4_2, X_4_3, X_4_4], axis=0, ignore_index=False)
X_4.to_csv('X_4.csv')

In [ ]:
np.random.seed(seed_number)
X_4_1 = pd.DataFrame(0, index = range(1, M_1_1+1), columns = range(1)).astype(float)
for i in range (1, M_1_1+1):
    X_4_1[0][i] = np.random.gamma(k_3 * (ILI_info.ILI_time[3] - np.random.uniform(0, ILI_info.ILI_time[0]))\
                                  ** nu_3, scale=ScalePar_3) 

X_4_2 = pd.DataFrame(0, index = range(1, M_2 - M_1_1+1), columns = range(1)).astype(float)
for i in range (1, M_2 - M_1_1+1):
    X_4_2[0][i] = np.random.gamma(k_3 * (ILI_info.ILI_time[3] - np.random.uniform(ILI_info.ILI_time[0],\
                                   ILI_info.ILI_time[1])) ** nu_3, scale=ScalePar_3) 

X_4_3 = pd.DataFrame(0, index = range(1, M_3 - M_2+1), columns = range(1)).astype(float)
for i in range (1, M_3 - M_2+1):
    X_4_3[0][i] = np.random.gamma(k_3 * (ILI_info.ILI_time[3] - np.random.uniform(ILI_info.ILI_time[1],\
                                   ILI_info.ILI_time[2])) ** nu_3, scale=ScalePar_3) 

X_4_4 = pd.DataFrame(0, index = range(1, M_4_4+1), columns = range(1)).astype(float)
for i in range (1, M_4_4+1):
    X_4_4[0][i] = np.random.gamma(k_3 * (ILI_info.ILI_time[3] - np.random.uniform(ILI_info.ILI_time[2],\
                                   ILI_info.ILI_time[3])) ** nu_3, scale=ScalePar_3) 


X_4 = pd.concat([X_4_1, X_4_2, X_4_3, X_4_4], axis=0, ignore_index=False)
X_4.to_csv('X_4.csv')

In [ ]:
# Visual comparison between actual and estimated depth for X_4
plt.figure(figsize=(12, 9))
plt.xlabel('Pit Depth (mm)', fontsize = 20)
plt.ylabel('PMF', fontsize = 20)
bins_start_X_4 = np.int(np.min([np.min(pd.concat([Prediction_Actual_Depth[0], Initiation_3_Actual_Depth[1],\
                                                Initiation_2_Actual_Depth[2], Initiation_1_Actual_Depth[3]])),\
                              np.min(X_4[0])]))
bins_end_X_4   = np.int(np.max([np.max(pd.concat([Prediction_Actual_Depth[0], Initiation_3_Actual_Depth[1],\
                                                Initiation_2_Actual_Depth[2], Initiation_1_Actual_Depth[3]])),\
                              np.max(X_4[0])]))
plt.hist(pd.concat([Prediction_Actual_Depth[0], Initiation_3_Actual_Depth[1], Initiation_2_Actual_Depth[2],\
                    Initiation_1_Actual_Depth[3]]), bins_no, color = 'green', range=(bins_start_X_4, bins_end_X_4),\
         histtype='step', density=True, linewidth=2, label='Simulated actual depth (X_4) at t_4');
plt.hist(X_4[0], bins_no, color = 'red', range=(bins_start_X_4, bins_end_X_4), histtype='step', density=True,\
         linewidth=1, label='Estimated actual depth (X_4) at t_4');

plt.xlim([bins_start_X_4, 30]);
plt.title('' , fontsize = 16)
plt.legend(loc='upper right');

In [ ]:
# Calculating number of pits at each bin of the PMF of actual depth and estimated depth without POD
(actual_N_4, bins_4, patches) = plt.hist(pd.concat([Prediction_Actual_Depth[0], Initiation_3_Actual_Depth[1],\
                                                    Initiation_2_Actual_Depth[2], Initiation_1_Actual_Depth[3]]),\
                                         bins_no, range=(bins_start_X_4, bins_end_X_4), histtype='step', density=False,\
                                         linewidth=2, color = 'blue', label='Distribution of actual samples ILI_3');
(estimated_N_WO_POD_4, bins_4, patches4) = plt.hist(X_4[0], bins_no,range=(bins_start_X_4, bins_end_X_4), \
                                                 histtype='step', density=False, linewidth=2, color = 'red',\
                                                 label='Distribution of estimated depth of detected pits');


In [ ]:
N_Actual_4 = pd.concat([Prediction_Actual_Depth[0], Initiation_3_Actual_Depth[1],\
                Initiation_2_Actual_Depth[2], Initiation_1_Actual_Depth[3]]).shape[0]
N_Estimated_4 = X_4.shape[0] 
KLD4 = KL_divergences(actual_N_4/N_Actual_4, estimated_N_WO_POD_4/N_Estimated_4, bins_no)
KLD4

In [ ]:
SKLD4 = (KL_divergences(actual_N_4/N_Actual_4, estimated_N_WO_POD_4/N_Estimated_4, bins_no) + \
         KL_divergences(estimated_N_WO_POD_4/N_Estimated_4, actual_N_4/N_Actual_4, bins_no))/2
SKLD4

In [ ]:
ChiSq_4 = 0
for i in range(0, bins_no):
    if actual_N_4[i] > 0 or estimated_N_WO_POD_4[i] > 0:
        ChiSq_4 = ChiSq_4 + (actual_N_4[i]/N_Actual_4 - estimated_N_WO_POD_4[i]/N_Estimated_4)**2\
        /(actual_N_4[i]/N_Actual_4 + estimated_N_WO_POD_4[i]/N_Estimated_4)
ChiSq_4

In [ ]:
Results = pd.DataFrame(0, index = range(1, 10), columns = range(4)).astype(float)
Results.columns = ['assumed', 'ILI_1', 'ILI_2', 'ILI_3']
Results.index = ['k', 'nu', 'ScalePar', 'OB: alpha_1_1', 'OB: alpha_2_1', 'OB: alpha_2_2', 'OB: alpha_3_1',\
                 'OB: alpha_3_2', 'OB: alpha_3_3']
Results.assumed[0] = k
Results.assumed[1] = nu
Results.assumed[2] = ScalePar
Results.assumed[3] = alpha_1_1_actual
Results.assumed[4] = alpha_2_1_actual
Results.assumed[5] = alpha_2_2_actual
Results.assumed[6] = alpha_3_1_actual
Results.assumed[7] = alpha_3_2_actual
Results.assumed[8] = alpha_3_3_actual
Results.ILI_1[0] = k_1
Results.ILI_1[1] = nu_1
Results.ILI_1[2] = ScalePar_1
Results.ILI_1[3] = alpha_1_1_Est
Results.ILI_2[0] = k_2
Results.ILI_2[1] = nu_2
Results.ILI_2[2] = ScalePar_2
Results.ILI_2[4] = alpha_2_1_Est
Results.ILI_2[5] = alpha_2_2_Est 
Results.ILI_3[0] = k_3
Results.ILI_3[1] = nu_3
Results.ILI_3[2] = ScalePar_3
Results.ILI_3[6] = alpha_3_1_Est
Results.ILI_3[7] = alpha_3_2_Est
Results.ILI_3[8] = alpha_3_3_Est

Results

In [ ]:
ILI_info.Estimated_Pit_No[0] = M_1_1
ILI_info.Estimated_Pit_No[1] = M_2 - M_1_1
ILI_info.Estimated_Pit_No[2] = M_3 - M_2
ILI_info.Estimated_Pit_No[3] = M_4_4
ILI_info.KLD[0] = np.round(KLD1,2)
ILI_info.KLD[1] = KLD2
ILI_info.KLD[2] = KLD3
ILI_info.KLD[3] = KLD4
ILI_info.ChiSq[0] = ChiSq_1
ILI_info.ChiSq[1] = ChiSq_2
ILI_info.ChiSq[2] = ChiSq_3
ILI_info.ChiSq[3] = ChiSq_4
ILI_info

In [ ]:
Pit_generation_rate

In [ ]:
HPP_rate

In [ ]:
def PosNormal(mean, sigma):
    x = np.random.normal(mean,sigma)
    return(x if x>=0 else PosNormal(mean,sigma))

In [ ]:
def CumulativeHist (bins_No, Freqs):
    CH = np.zeros (bins_No)
    for i in range(1, bins_No):
        CH[i] = CH[i-1] + Freqs[i-1]
    return(CH)

In [ ]:
def DiscMCS(CumulHist, bins, bins_no):
    rand = np.random.uniform(0,1)*CumulHist[bins_no-1]
    for i in range(1, bins_no):
        if rand > CumulHist[i-1] and rand < CumulHist[i]:
            sample = ((bins[i]-bins[i-1])*rand-bins[i]*CumulHist[i-1]+bins[i-1]*CumulHist[i])/\
            (CumulHist[i] - CumulHist[i-1])
    return(sample)   

In [ ]:
CumuHist_X_4 = CumulativeHist(bins_no, estimated_N_WO_POD_4)

In [ ]:
np.random.seed(seed_number)
MCSIterNo = 10000
Operating_pressure_mean = 6.66       # MPa
Operating_pressure_STD  = Operating_pressure_mean * 0.02 # MPa
# Calculating probability of small leak at each time
prob_of_failure = pd.DataFrame(0, index=range(1, 1), columns=range(4)).astype(float)
prob_of_failure.columns = ["Safe", "Small Leak", "Large Leak", "Rupture"]
OperPressSamples = pd.DataFrame(0, index=range(0, MCSIterNo), columns=range(1)).astype(float)
BurstPressSamples = pd.DataFrame(0, index=range(1), columns=range(1)).astype(float)
RupturePressSamples = pd.DataFrame(0, index=range(1), columns=range(1)).astype(float)
PWTSamples = pd.DataFrame(0, index=range(0, MCSIterNo), columns=range(1)).astype(float)
X_4_sample = np.zeros(MCSIterNo)
for i in range(0, MCSIterNo):
    X_4_sample[i] = DiscMCS(CumuHist_X_4, bins_4, bins_no)

## Samplin from a histogram X_4
# Cumalative histogram CX_4
pit_length_mean = 15 # mm
pit_length_STD  = 3 # mm
small_leak_threshold = 0.8 * PWT # 0.8 is the common value in the literature
ultimate_tensile_strength_mean = 456  # MPa
ultimate_tensile_strength_STD = ultimate_tensile_strength_mean * 0.03  # MPa
epsilon = 1
outer_diameter = 506  # mm

N_small   = 0
N_large   = 0
N_rupture = 0
Counter = 0
for k in range(0, MCSIterNo):       
    pit_length = PosNormal(pit_length_mean, pit_length_STD)
    UTS = PosNormal(ultimate_tensile_strength_mean, ultimate_tensile_strength_STD)
    Op_pressure = PosNormal(Operating_pressure_mean, Operating_pressure_STD)
    PWTSample1 = PosNormal(Operating_pressure_mean, Operating_pressure_STD)
    PWTSample = PWT
    OperPressSamples.loc[k,0] = Op_pressure    
    PWTSamples.loc[k,0] = PWTSample1
    f1 = small_leak_threshold - X_4_sample[k]
    if f1 <= 0 :#and f2 > 0:
        N_small  = N_small + 1
    else:
        # burst pressure
        Pb = epsilon * UTS * PWTSample / outer_diameter * (1 - X_4_sample[k]/ PWTSample *\
            (1- np.exp(-0.157 * pit_length / np.sqrt(outer_diameter*(PWTSample - X_4_sample[k]) / 2))))
        if pit_length**2 / (outer_diameter * PWTSample) <= 50:
            M = np.sqrt(1+0.6275 * pit_length**2/(outer_diameter * PWTSample)-0.003375*pit_length**4 \
                        / (outer_diameter * PWTSample)**2)
        else:
            M =  3.3 + 0.032 * pit_length**2/(outer_diameter * PWTSample) 
        P_rupture = 2 * PWTSample * 0.9 * UTS / (M * outer_diameter)        
        BurstPressSamples.loc[Counter,0] = Pb
        RupturePressSamples.loc[Counter,0] = P_rupture
        Counter = Counter + 1        
        f2 = Pb - Op_pressure
        f3 = P_rupture - Op_pressure

        if f2 <= 0 and f3 > 0:
            N_large  = N_large + 1
        if f2 <= 0 and f3 <= 0:
            N_rupture  = N_rupture + 1 
prob_of_failure.loc[1, 'Small Leak'] = N_small / MCSIterNo
prob_of_failure.loc[1, 'Large Leak'] = N_large / MCSIterNo
prob_of_failure.loc[1, 'Rupture']    = N_rupture / MCSIterNo
prob_of_failure.loc[1, 'Safe'] = (MCSIterNo - N_small - N_large - N_rupture) / MCSIterNo

In [ ]:
prob_of_failure

In [ ]:
# Some characteristics for plots
import matplotlib.pyplot as plt
plt.rc('legend',**{'fontsize':20})
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20) 

In [ ]:
plt.style.available

In [ ]:
plt.style.use('seaborn-white')

In [ ]:
plt.style.use('classic')

In [ ]:
plt.figure(figsize=(12, 9))
plt.xlabel('Pressure(MPa)', fontsize = 20)
plt.ylabel('PMF', fontsize = 20)
# bins_start_X_1 = np.int(np.min([np.min(Initiation_1_Actual_Depth[0]), np.min(X_1[0])]))
# bins_end_X_1   = np.int(np.max([np.max(Initiation_1_Actual_Depth[0]), np.max(X_1[0])]))
plt.hist(OperPressSamples[0], 100, color = 'green', \
         histtype='step', density=True,  linewidth=2, label='Operating Pressure');
plt.hist(BurstPressSamples[0], 100, color = 'red', \
          histtype='step', density=True, linewidth=2, label='Burst Pressure');

plt.hist(RupturePressSamples[0], 100, color = 'blue', \
          histtype='step', density=True, linewidth=2, label='Rupture Pressure');

plt.title('' , fontsize = 20)
plt.legend(loc='upper center',)
